In [2]:
import io, os, zipfile, tempfile, requests
from pathlib import Path

import geopandas as gpd
from tqdm import tqdm

# --- 1.  Download the counties shapefile (TIGER 2023)  ------------------------
TIGER_URL = (
    "https://www2.census.gov/geo/tiger/TIGER2023/COUNTY/tl_2023_us_county.zip"
)

cache_dir = Path.home() / ".cache" / "tiger"
cache_dir.mkdir(parents=True, exist_ok=True)
zip_path = cache_dir / TIGER_URL.split("/")[-1]

if not zip_path.exists():
    print("Downloading TIGER counties shapefile…")
    r = requests.get(TIGER_URL, stream=True)
    r.raise_for_status()
    total = int(r.headers.get("content-length", 0))
    with open(zip_path, "wb") as f, tqdm(
        total=total, unit="B", unit_scale=True, desc="  "
    ) as pbar:
        for chunk in r.iter_content(chunk_size=1 << 15):
            f.write(chunk)
            pbar.update(len(chunk))

# --- 2.  Read the shapefile directly from the ZIP ----------------------------
with zipfile.ZipFile(zip_path) as z:
    with tempfile.TemporaryDirectory() as tmp:
        z.extractall(tmp)
        shp = next(Path(tmp).glob("*.shp"))
        counties = gpd.read_file(shp)

# --- 3.  Keep only California -------------------------------------------------
ca_counties = counties[counties["STATEFP"] == "06"].copy()

# --- 4.  Re-project to an equal-area CRS, compute centroids -------------------
#   EPSG:3310 = NAD83 / California Albers
ca_counties_aea = ca_counties.to_crs(epsg=3310)
ca_counties["centroid"] = (
    ca_counties_aea.geometry.centroid.to_crs(epsg=4326)
)

# --- 5.  Build a neat results frame ------------------------------------------
out = ca_counties[["NAME", "centroid"]].copy()
out["latitude"] = out.centroid.y
out["longitude"] = out.centroid.x
out = out.drop(columns="centroid").sort_values("NAME")

# --- 6.  Use the data ---------------------------------------------------------
print(out.to_markdown(index=False, floatfmt=".6f"))

| NAME            |   latitude |   longitude |
|:----------------|-----------:|------------:|
| Alameda         |  37.650583 | -121.917803 |
| Alpine          |  38.597098 | -119.820603 |
| Amador          |  38.446604 | -120.651663 |
| Butte           |  39.666554 | -121.600701 |
| Calaveras       |  38.204460 | -120.554664 |
| Colusa          |  39.177508 | -122.236735 |
| Contra Costa    |  37.923543 | -121.950945 |
| Del Norte       |  41.744691 | -123.957661 |
| El Dorado       |  38.778906 | -120.525038 |
| Fresno          |  36.758554 | -119.653399 |
| Glenn           |  39.598481 | -122.392014 |
| Humboldt        |  40.702760 | -123.916299 |
| Imperial        |  33.039935 | -115.365348 |
| Inyo            |  36.510176 | -117.405636 |
| Kern            |  35.343561 | -118.729046 |
| Kings           |  36.074961 | -119.815836 |
| Lake            |  39.099482 | -122.752781 |
| Lassen          |  40.672632 | -120.593282 |
| Los Angeles     |  34.196108 | -118.261843 |
| Madera     